In [7]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

ACCOUNT_BUNDLER_LIST = dataiku.Dataset("ACCOUNT_BUNDLER_LIST")
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST.get_dataframe()
print(len(ACCOUNT_BUNDLER_LIST_df))

NAFCUSTOMER_RDW_CONVERSIONS = dataiku.Dataset("NAFCUSTOMER_RDW_CONVERSIONS")
NAFCUSTOMER_RDW_CONVERSIONS_df = NAFCUSTOMER_RDW_CONVERSIONS.get_dataframe()
print(len(NAFCUSTOMER_RDW_CONVERSIONS_df))

# Read recipe inputs
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER = dataiku.Dataset("NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER")
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df = NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER.get_dataframe()
print(len(NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df))

274
2170800
4806903


In [56]:
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT
0,9100004251546,CAROLINA AIR CARE,WEX FLEET UNIVERSAL PRIMARY,2017-04-27 00:00:00.000,SIEBEL,2017-04-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2017,4,15
1,417008246662,U S D 315 COLBY PUBLIC SCHOOLS,U S D 315 COLBY PUBLIC SCHOOLS,2010-05-14 00:00:00.000,TANDEM,2010-05-14 00:00:00.000,2022-09-10 00:00:00.000,NNCL Conversion,04,Account Conversion,2018,2,14
2,430004184602,EASTER SEALS NEW HAMPSHIRE,EASTER SEALS NEW HAMPSHIRE,2000-08-02 00:00:00.000,TANDEM,2000-08-02 00:00:00.000,2022-09-17 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,4,121
3,9100004755591,11300 GRANT COUNTY WA,AMR HOLDCO INC,2018-02-14 00:00:00.000,SIEBEL,2018-02-14 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2019,4,11
4,444007634488,MARTIN CHRYSLER LLC,MARTIN CHRYSLER LLC,2007-12-04 00:00:00.000,TANDEM,2007-12-04 00:00:00.000,2022-04-19 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,2,9


In [58]:
df_a = NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df
df_a['CUSTOMER_ACCOUNT_ID'] = df_a['CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')
df_a['CUSTOMER_ACCOUNT_NAME'] = df_a['CUSTOMER_ACCOUNT_NAME'].str.upper()
df_a['EDW_CUSTOMER_NAME'] = df_a['EDW_CUSTOMER_NAME'].str.upper()

ACCOUNT_BUNDLER_LIST_df['IS_BUNDLER'] = True
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST_df[['EDW_CUSTOMER_NAME','IS_BUNDLER']]
ACCOUNT_BUNDLER_LIST_df.head()

df = pd.merge(df_a,ACCOUNT_BUNDLER_LIST_df, how='left', on='EDW_CUSTOMER_NAME')
df.loc[df["IS_BUNDLER"].isnull(),'IS_BUNDLER'] = False
df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT,IS_BUNDLER
0,9100004251546,CAROLINA AIR CARE,WEX FLEET UNIVERSAL PRIMARY,2017-04-27 00:00:00.000,SIEBEL,2017-04-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2017,4,15,True
1,417008246662,U S D 315 COLBY PUBLIC SCHOOLS,U S D 315 COLBY PUBLIC SCHOOLS,2010-05-14 00:00:00.000,TANDEM,2010-05-14 00:00:00.000,2022-09-10 00:00:00.000,NNCL Conversion,04,Account Conversion,2018,2,14,False
2,430004184602,EASTER SEALS NEW HAMPSHIRE,EASTER SEALS NEW HAMPSHIRE,2000-08-02 00:00:00.000,TANDEM,2000-08-02 00:00:00.000,2022-09-17 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,4,121,False
3,9100004755591,11300 GRANT COUNTY WA,AMR HOLDCO INC,2018-02-14 00:00:00.000,SIEBEL,2018-02-14 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2019,4,11,False
4,444007634488,MARTIN CHRYSLER LLC,MARTIN CHRYSLER LLC,2007-12-04 00:00:00.000,TANDEM,2007-12-04 00:00:00.000,2022-04-19 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,2,9,False


In [59]:
df['CUSTOMER'] = np.nan
df['CUST_CALC_SOURCE'] = 'Unknown'
df.loc[df.IS_BUNDLER,'EDW_CUSTOMER_NAME'] = np.nan

df.loc[~df['EDW_CUSTOMER_NAME'].isnull(),'CUSTOMER'] = df["EDW_CUSTOMER_NAME"]
df.loc[~df['EDW_CUSTOMER_NAME'].isnull(),'CUST_CALC_SOURCE'] = 'EDW'

df.loc[df['CUSTOMER'].isnull(),'CUST_CALC_SOURCE'] = 'ACCOUNT'
df.loc[df['CUSTOMER'].isnull(),'CUSTOMER'] = df.CUSTOMER_ACCOUNT_NAME


ending_tokens = [' 2', ' 3', ' 4', ' 04', ' 5', ' 6', ' 7', ' 8', ' 9',' (2)',
                 ' (3)',' (04)',' (4)', ' (5)', ' (6)', ' (7)', ' (8)',
                 ' (9)',' (25)','  (32)', ' AD', ' LD', 'L1']

df['CUSTOMER'].str.strip()

for s in ending_tokens:
    index_offset = -1*(len(s))
    df.loc[df['CUSTOMER'].str.endswith(s, na=False),"CUSTOMER"] = df['CUSTOMER'].str[:index_offset]


In [87]:
df_account_with_customer = df[['CUSTOMER_ACCOUNT_ID','CUSTOMER']].copy()
df_account_with_customer = df_account_with_customer.drop_duplicates(subset='CUSTOMER_ACCOUNT_ID')
print(len(df_account_with_customer))
df_account_with_customer.head()

698399


,CUSTOMER_ACCOUNT_ID,CUSTOMER
0,9100004251546,CAROLINA AIR CARE
1,417008246662,U S D 315 COLBY PUBLIC SCHOOLS
2,430004184602,EASTER SEALS NEW HAMPSHIRE
3,9100004755591,AMR HOLDCO INC
4,444007634488,MARTIN CHRYSLER LLC


In [54]:
df.CUST_CALC_SOURCE.value_counts(dropna=False)

EDW        2707344
ACCOUNT    2099559
Name: CUST_CALC_SOURCE, dtype: int64

In [63]:
df_g = df.groupby(['CUSTOMER']).CUSTOMER_ACCOUNT_ID.nunique().reset_index()
df_g.columns = ['CUSTOMER','ACCOUNT_COUNT']
df_g = df_g.sort_values(by=['ACCOUNT_COUNT'], ascending=False)
df_g.head(100)

,CUSTOMER,ACCOUNT_COUNT
376404,OREILLY AUTOMOTIVE INC,5798
424103,RENTACENTER,2606
125165,COMMONWEALTH OF PENNSYLVANIA,2277
475675,STATE OF NEW YORK,1672
190651,FERGUSON ENTERPRISES,1627
475650,STATE OF ILLINOIS,1472
179967,ENTERPRISE XOM DUMP,1431
443372,SAFELITE GROUP INC,1283
16957,AARON'S LLC,1247
179677,ENTERPRISE FM DUMP,1145


In [64]:
NAFCUSTOMER_RDW_CONVERSIONS_df.head()
NAFCUSTOMER_RDW_CONVERSIONS_df.STATUS_DATE.min()

'1901-01-01 00:00:00.000'

In [84]:
NAFCUSTOMER_RDW_CONVERSIONS_df.head()
df_conv = NAFCUSTOMER_RDW_CONVERSIONS_df[['FLEET_ID','CLASSIC_ACCOUNT_NUMBER','FLEET_NAME']].copy()
print(len(df_conv))
df_conv = df_conv[~df_conv.CLASSIC_ACCOUNT_NUMBER.isnull()]
print(len(df_conv))

df_conv.columns = ['CUSTOMER_ACCOUNT_ID', 'CLASSIC_CUSTOMER_ACCOUNT_ID', 'FLEET_NAME']
df_conv['CUSTOMER_ACCOUNT_ID'] = df_conv['CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')
df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'] = pd.to_numeric(df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'], errors='coerce')
df_conv = df_conv[~df_conv.CLASSIC_CUSTOMER_ACCOUNT_ID.isnull()]
df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'] = df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'].astype('int64', errors='ignore')
df_conv.head()

2170800
743702


,CUSTOMER_ACCOUNT_ID,CLASSIC_CUSTOMER_ACCOUNT_ID,FLEET_NAME
6,7000000006560,1820,GRANVILLE EXMPTED VILL
43,7000000010283,70627,WEST GROVE FIRE CO
49,9100000678024,12469,biggie rat plumbing
57,7000000009075,85799,QUALITY COLLISION INC
59,7000000010082,47575,DEMARCO-STONE FUNERAL


In [89]:
print(len(df_conv))
df_conv = pd.merge(df_conv, df_account_with_customer, on='CUSTOMER_ACCOUNT_ID', how='inner')
print(len(df_conv))

651050
9460


In [91]:
df_conv.head(100)

,CUSTOMER_ACCOUNT_ID,CLASSIC_CUSTOMER_ACCOUNT_ID,FLEET_NAME,CUSTOMER
0,9100001042479,870232105,Benedictine Sisters,BENEDICTINE SISTERS
1,9100001041237,870191343,Dube Water Trucks Inc.,DUBE WATER TRUCKS INC
2,9100005351139,496003801305,Maryland Heights Mo,MARYLAND HEIGHTS MO
3,9100001038852,870183829,Hawaiian Pa'akai Inc.,HAWAIIAN PA'AKAI INC
4,9100007613460,496006076004,Houston-Highway 90,HOUSTONHIGHWAY 90
5,400016523,400016523,Yukon Equip. Inc.,YUKON EQUIP INC
6,9100007154937,463006115331,EPISCOPAL HOME COMMUNITIES THE,EPISCOPAL HOME COMMUNITIES THE
7,7000000010286,70668,Lancaster Parking Auth.,LANCASTER PARKING AUTH.
8,7000000006759,5102,MIRACLE ELECTRICAL SERVIC,MIRACLE ELECTRICAL SERVIC
9,9100004133664,446410,Damico Catering,DAMICO CATERING


In [0]:
# Write recipe outputs
#NAFCUSTOMER_LONG_HISTORY_MATCHES = dataiku.Dataset("NAFCUSTOMER_LONG_HISTORY_MATCHES")
#NAFCUSTOMER_LONG_HISTORY_MATCHES.write_with_schema(NAFCUSTOMER_LONG_HISTORY_MATCHES_df)